In [ ]:
import pandas as pd

# Replace 'your_file.bedGraph' with the path to your actual bedGraph file
file_path = 'data/dataset1/chip-seq_files/Galaxy7-[bigWigToBedGraph on data 3].bedgraph'

# Read the bedGraph file into a DataFrame
df = pd.read_csv(file_path, sep='\t', header=None, comment='#')

# Optionally, assign column names
df.columns = ['chrom', 'start', 'end', 'value']

# Show the first few rows
print(df.head())


In [ ]:
import pandas as pd

# Path to your BEDGRAPH file
chip_path = "data/dataset2/chip-seq_files/"

# Read the file into a DataFrame
chip_df = pd.read_csv(
    chip_path,
    sep="\t",
    header=None,
    names=["chrom", "start", "end", "signal"],
    dtype={"chrom": str, "start": int, "end": int, "signal": float}
)

# Show the first 10 rows to inspect the data
print(chip_df.head(10))

# Optional: check for missing or malformed rows
print("\nMissing values in each column:")
print(chip_df.isna().sum())




  chrom  start  end    signal
0     1      0   40  0.000000
1     1     40   70  0.615326
2     1     70  190  1.230650
3     1    190  225  0.615326
4     1    225  245  0.000000
5     1    245  335  0.615326
6     1    335  365  1.230650
7     1    365  370  1.845980
8     1    370  380  3.691960
9     1    380  390  4.307280

Missing values in each column:
chrom     0
start     0
end       0
signal    0
dtype: int64


In [ ]:
chip_df["chrom"].unique()

array(['1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'B73V4_ctg1',
       'B73V4_ctg10', 'B73V4_ctg100', 'B73V4_ctg101', 'B73V4_ctg102',
       'B73V4_ctg103', 'B73V4_ctg104', 'B73V4_ctg105', 'B73V4_ctg106',
       'B73V4_ctg107', 'B73V4_ctg108', 'B73V4_ctg109', 'B73V4_ctg11',
       'B73V4_ctg110', 'B73V4_ctg111', 'B73V4_ctg112', 'B73V4_ctg113',
       'B73V4_ctg114', 'B73V4_ctg115', 'B73V4_ctg116', 'B73V4_ctg117',
       'B73V4_ctg118', 'B73V4_ctg119', 'B73V4_ctg12', 'B73V4_ctg120',
       'B73V4_ctg121', 'B73V4_ctg122', 'B73V4_ctg123', 'B73V4_ctg124',
       'B73V4_ctg125', 'B73V4_ctg126', 'B73V4_ctg127', 'B73V4_ctg128',
       'B73V4_ctg129', 'B73V4_ctg13', 'B73V4_ctg130', 'B73V4_ctg131',
       'B73V4_ctg132', 'B73V4_ctg133', 'B73V4_ctg134', 'B73V4_ctg135',
       'B73V4_ctg136', 'B73V4_ctg137', 'B73V4_ctg138', 'B73V4_ctg139',
       'B73V4_ctg14', 'B73V4_ctg140', 'B73V4_ctg141', 'B73V4_ctg142',
       'B73V4_ctg143', 'B73V4_ctg144', 'B73V4_ctg145', 'B73V4_ctg146',
       'B7

In [ ]:
import pandas as pd
import numpy as np
# === Load BEDGRAPH file ===
chip_path = "data/dataset1/chip-seq_files/Galaxy7-[bigWigToBedGraph on data 3].bedgraph"
chip_df = pd.read_csv(
    chip_path,
    sep="\t",
    header=None,
    names=["chrom", "start", "end", "signal"],
    dtype={"chrom": str}  # make sure chromosome is treated as string
)

# === Keep only chromosomes "1" to "10" ===
chroms_to_keep = [str(i) for i in range(1, 11)]
filtered_chip_df = chip_df[chip_df["chrom"].isin(chroms_to_keep)]

# === Save to CSV ===
filtered_chip_df.to_csv("chipseq_chr1_to_10.csv", index=False)

print("Saved chromosomes 1 to 10 to 'chipseq_chr1_to_10.csv'")


Saved chromosomes 1 to 10 to 'chipseq_chr1_to_10.csv'


In [ ]:
import os
import gzip
import pandas as pd
import numpy as np
from urllib.parse import unquote
from Bio import SeqIO
dna_dir = "data/dataset1/dna_sequence"
gff3_dir = "data/dataset1/gff3_files"
#
# # === Collect all FASTA and GFF3 files ===
fasta_files = sorted([
     os.path.join(dna_dir, f) for f in os.listdir(dna_dir)
     if f.lower().endswith(".fa.gz")
 ])
gff3_files = sorted([
     os.path.join(gff3_dir, f) for f in os.listdir(gff3_dir)
     if f.lower().endswith(".gff3")
])
#
#
# === Parse GFF3 attributes ===
def parse_attributes(attr_str):
    attr_dict = {}
    for pair in attr_str.strip().split(";"):
        if "=" in pair:
            key, value = pair.split("=", 1)
            attr_dict[key.strip()] = unquote(value.strip())
    return attr_dict
#
#
# # === Function to parse GFF3 and extract gene entries for a given chromosome ===
def parse_gff3(gff3_file, chrom_id):
    genes = []
    with open(gff3_file, encoding="utf-8") as f:
        for line in f:
            if line.startswith("#"):
                continue
            parts = line.strip().split("\t")
            if len(parts) < 9:
                continue
            if parts[2] != "gene":
                continue
            if parts[0] != chrom_id:
                continue  # skip if this line refers to a different chromosome

            start = int(parts[3]) - 1  # Convert to 0-based index
            end = int(parts[4])
            strand = parts[6]
            attrs = parse_attributes(parts[8])
            gene_id = attrs.get("ID", "NA")

            genes.append((start, end, strand, gene_id))
    return genes


# === Extract gene sequences ===
gene_sequences = []

for fasta_path, gff_path in zip(fasta_files, gff3_files):
    print(f"Processing: {os.path.basename(fasta_path)} with {os.path.basename(gff_path)}")

    # Read the chromosome sequence (support gzipped or uncompressed)
    if fasta_path.endswith(".gz"):
         with gzip.open(fasta_path, "rt", encoding="utf-8") as f:
             record = next(SeqIO.parse(f, "fasta"))
    else:
         with open(fasta_path, "r", encoding="utf-8") as f:
            record = next(SeqIO.parse(f, "fasta"))

    chrom_seq = record.seq
    chrom_id = record.id

    # Parse gene entries from the GFF3 file
    genes = parse_gff3(gff_path, chrom_id)

    for start, end, strand, gene_id in genes:
        # Boundary check to avoid errors
        if start < 0 or end > len(chrom_seq):
            continue
#
        gene_seq = chrom_seq[start:end]
        if strand == "-":
            gene_seq = gene_seq.reverse_complement()

        gene_sequences.append({
            "gene_id": gene_id,
            "chrom": chrom_id,
            "start": start,
            "end": end,
            "strand": strand,
            "sequence": str(gene_seq)
        })
#
# === Convert to DataFrame ===
df_genes = pd.DataFrame(gene_sequences)

# === Preview first few entries ===
print(df_genes.head())
geo_path = "data/dataset1/alias/genes_to_alias_ids.tsv"
df = pd.read_csv(geo_path, sep='\t')
alias_path = "data/dataset1/geo_file/abundance.tsv"
df_alias = pd.read_csv(alias_path, sep='\t')
df.rename(columns={"Zm00001eb000010": "id1", "B73 Zm00001eb.1": "id2", "Zm00001d027230": "gene_alias_id"})

import pandas as pd
#
# # Fix column names in 'df'
df.columns = ['id1', 'id2', 'gene_alias_id', 'AGPv4_Zm00001d.2']
#
# # Step 1: Clean the 'gene_id' column in df_genes
df_genes['gene_id_clean'] = df_genes['gene_id'].str.replace('gene:', '', regex=False)
#
# # Step 2: Create a mapping from 'id1' to 'gene_alias_id'
id_to_alias = df.set_index('id1')['gene_alias_id'].to_dict()
#
# # Step 3: Map gene_alias_id to df_genes
df_genes['alias_id'] = df_genes['gene_id_clean'].map(id_to_alias)
#
# # Step 4: Clean up the temporary column
df_genes.drop(columns=['gene_id_clean'], inplace=True)
#
# # Check the results
print(df_genes[['gene_id', 'alias_id']].head(50))
#
# # Drop rows where alias_id is NaN
df_genes = df_genes.dropna(subset=['alias_id'])
#
# # Reset index if needed
df_genes = df_genes.reset_index(drop=True)
#
# # Optional: check that it's gone
print(df_genes['alias_id'].isna().sum())
#

#
# # Step 1: Clean up df_alias to remove transcript suffix
df_alias['clean_id'] = df_alias['target_id'].str.replace(r'_T\d+$', '', regex=True)
#
# # Step 2: Group by clean_id and sum or average TPM if needed (in case multiple transcripts per gene)
# # Here we'll sum TPMs for all isoforms of a gene
tpm_by_gene = df_alias.groupby('clean_id')['tpm'].sum().reset_index()
#
# # Step 3: Merge df_genes with this TPM info using alias_id == clean_id
df_genes = df_genes.merge(tpm_by_gene, how='left', left_on='alias_id', right_on='clean_id')
#
# # Step 4: Rename the column to tpm_value and drop clean_id
df_genes = df_genes.rename(columns={'tpm': 'tpm_value'}).drop(columns=['clean_id'])
#
# # Done
print(df_genes.head())
#
# # Mapping dictionary
base_map = {'A': 0, 'T': 1, 'G': 2, 'C': 3}
#
#
# # Function to encode a DNA sequence string
def encode_sequence(seq):
     return [base_map.get(base, -1) for base in seq.upper()]  # -1 for unknown bases like N
#
#
# # Apply to each row in df_genes['sequence']
df_genes['encoded_sequence'] = df_genes['sequence'].apply(encode_sequence)
#
# # Done
print(df_genes[['sequence', 'encoded_sequence']].head())
#
df_genes.drop("sequence", axis=1, inplace=True)
#
max_len = df_genes['encoded_sequence'].apply(len).max()
print("Maximum encoded sequence length:", max_len)
#
# # Step 1: Store lengths of all encoded sequences
sequence_lengths = df_genes['encoded_sequence'].apply(len)
#
# # Step 2: Count how many are greater than 50,000
num_greater_than_40000 = (sequence_lengths > 1000).sum()
#
# # Output results
print("Total sequences:", len(sequence_lengths))
print("Sequences > 50,000 bases:", num_greater_than_40000)
#
# Fixed length
FIXED_LEN = 3000
PAD_VALUE = 0  # A = 0

#
def pad_or_truncate(seq):
    if len(seq) > FIXED_LEN:
        return seq[:FIXED_LEN]
    else:
        return seq + [PAD_VALUE] * (FIXED_LEN - len(seq))  # pad
#
#
# # Apply to each sequence
df_genes['encoded_50k'] = df_genes['encoded_sequence'].apply(pad_or_truncate)

# Check shape of one example
print(len(df_genes['encoded_50k'].iloc[0]))
#
print(df_genes.head())

df_genes['strand'] = df_genes['strand'].map({'+': 1, '-': 0})

df_genes.rename(columns={"encoded_50k": "sequence"}, inplace=True)

df_genes["sequence"]
#

#
#
def fix_sequence(seq):
    return [4 if val == -1 else val for val in seq]
#

df_genes['sequence'] = df_genes['sequence'].apply(fix_sequence)
x = np.array(df_genes['sequence'].tolist(), dtype=np.uint8)



y = df_genes["tpm_value"]

print("this is x shape:", x.shape)
print("this is y shape:", y.shape)


Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.1.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.1.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.10.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.10.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.2.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.2.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.3.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.3.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.4.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.4.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.5.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.5.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.6.fa.gz with Zea_mays.Zm-B73-REFERENCE-NAM-5.0.60.chromosome.6.gff3
Processing: Zea_mays.Zm-B73-REFERENCE-NAM-5.0.dna.chromosome.7.fa.gz with Zea_mays.Zm-B73-REFER

In [ ]:
df_genes.head()

,gene_id,chrom,start,end,strand,alias_id,tpm_value,encoded_sequence,sequence
0,gene:Zm00001eb000020,1,41213,46762,0,Zm00001d027231,62.888229,"[1, 3, 1, 3, 0, 2, 2, 1, 1, 1, 2, 0, 0, 0, 3, ...","[1, 3, 1, 3, 0, 2, 2, 1, 1, 1, 2, 0, 0, 0, 3, ..."
1,gene:Zm00001eb000050,1,108553,114382,0,Zm00001d027234,0.000000,"[3, 1, 2, 3, 3, 2, 0, 2, 3, 0, 2, 1, 2, 2, 0, ...","[3, 1, 2, 3, 3, 2, 0, 2, 3, 0, 2, 1, 2, 2, 0, ..."
2,gene:Zm00001eb000060,1,188558,189581,0,Zm00001d027239,38.358540,"[3, 3, 0, 1, 1, 2, 3, 3, 3, 1, 0, 2, 3, 0, 3, ...","[3, 3, 0, 1, 1, 2, 3, 3, 3, 1, 0, 2, 3, 0, 3, ..."
3,gene:Zm00001eb000070,1,190191,198832,0,Zm00001d027240,4.318454,"[0, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, ...","[0, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, ..."
4,gene:Zm00001eb000080,1,200261,203393,0,Zm00001d027242,42.850347,"[1, 1, 2, 2, 0, 3, 3, 0, 1, 3, 1, 1, 1, 0, 1, ...","[1, 1, 2, 2, 0, 3, 3, 0, 1, 3, 1, 1, 1, 0, 1, ..."


In [ ]:
df_genes["encoded_sequence"].apply(len)


0         5549
1         5829
2         1023
3         8641
4         3132
         ...  
32281     2543
32282    12520
32283    23080
32284    34680
32285    23577
Name: encoded_sequence, Length: 32286, dtype: int64

In [ ]:
df_genes.drop(columns={"encoded_sequence"}, inplace=True)

In [ ]:
df_genes.head()

,gene_id,chrom,start,end,strand,alias_id,tpm_value,sequence
0,gene:Zm00001eb000020,1,41213,46762,0,Zm00001d027231,62.888229,"[1, 3, 1, 3, 0, 2, 2, 1, 1, 1, 2, 0, 0, 0, 3, ..."
1,gene:Zm00001eb000050,1,108553,114382,0,Zm00001d027234,0.000000,"[3, 1, 2, 3, 3, 2, 0, 2, 3, 0, 2, 1, 2, 2, 0, ..."
2,gene:Zm00001eb000060,1,188558,189581,0,Zm00001d027239,38.358540,"[3, 3, 0, 1, 1, 2, 3, 3, 3, 1, 0, 2, 3, 0, 3, ..."
3,gene:Zm00001eb000070,1,190191,198832,0,Zm00001d027240,4.318454,"[0, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, ..."
4,gene:Zm00001eb000080,1,200261,203393,0,Zm00001d027242,42.850347,"[1, 1, 2, 2, 0, 3, 3, 0, 1, 3, 1, 1, 1, 0, 1, ..."


In [ ]:
df_genes.to_csv("data.csv", index=False)

In [ ]:
import pandas as pd
data=pd.read_csv("chipseq_chr1_to_10.csv")
data.head()

,chrom,start,end,signal
0,1,0,40,0.000000
1,1,40,70,0.615326
2,1,70,190,1.230650
3,1,190,225,0.615326
4,1,225,245,0.000000


In [ ]:
import pandas as pd
import pyranges as pr

print("🔄 Step 1: Loading ChIP-Seq bedgraph file...")
chip_df = pd.read_csv("chipseq_chr1_to_10.csv")
print(f"✅ chip_df loaded: {chip_df.shape[0]} rows")

print("🔄 Step 2: Loading gene dataset...")
gene_df = pd.read_csv("data.csv")  # Make sure this file contains a 'gene_id' column
print(f"✅ gene_df loaded: {gene_df.shape[0]} rows")

# === Ensure chromosome columns are strings ===
print("🔄 Step 3: Preparing columns for PyRanges...")
chip_df["Chromosome"] = chip_df["chrom"].astype(str)
chip_df["Start"] = chip_df["start"]
chip_df["End"] = chip_df["end"]
chip_df["Signal"] = chip_df["signal"]

gene_df["Chromosome"] = gene_df["chrom"].astype(str)
gene_df["Start"] = gene_df["start"]
gene_df["End"] = gene_df["end"]

# Use a unique gene identifier
gene_df["GeneID"] = gene_df["gene_id"]  # Adjust if your column is named differently
print("✅ Columns renamed and cast to correct types.")

# === Convert to PyRanges objects ===
print("🔄 Step 4: Converting DataFrames to PyRanges...")
chip_ranges = pr.PyRanges(chip_df[["Chromosome", "Start", "End", "Signal"]])
gene_ranges = pr.PyRanges(gene_df[["Chromosome", "Start", "End", "GeneID"]])
print("✅ PyRanges objects created.")

# === Perform intersection ===
print("🔄 Step 5: Performing genomic join (intersection)...")
joined = gene_ranges.join(chip_ranges, strandedness=False)
print(f"✅ Join completed: {joined.df.shape[0]} overlapping records found")

# === Group by GeneID and calculate average ChIP signal ===
print("🔄 Step 6: Grouping and averaging ChIP signal for each gene...")
grouped = joined.df.groupby("GeneID", observed=True)["Signal"].mean().reset_index()
grouped.rename(columns={"Signal": "avg_chip_signal"}, inplace=True)
print(f"✅ Averaging complete: {grouped.shape[0]} genes with signal")

# === Merge back with original gene_df ===
print("🔄 Step 7: Merging signal data back into gene dataset...")
result = pd.merge(gene_df, grouped, on="GeneID", how="left")
result["avg_chip_signal"] = result["avg_chip_signal"].fillna(0)
print("✅ Merge complete.")

# === Save the combined dataset ===
print("💾 Saving final result to 'gene_dataset_with_chip.csv'...")
result.to_csv("gene_dataset_with_chip.csv", index=False)
print("🎉 Done! Combined dataset saved as 'gene_dataset_with_chip.csv'")


🔄 Step 1: Loading ChIP-Seq bedgraph file...
✅ chip_df loaded: 38671573 rows
🔄 Step 2: Loading gene dataset...
✅ gene_df loaded: 32286 rows
🔄 Step 3: Preparing columns for PyRanges...
✅ Columns renamed and cast to correct types.
🔄 Step 4: Converting DataFrames to PyRanges...
✅ PyRanges objects created.
🔄 Step 5: Performing genomic join (intersection)...
✅ Join completed: 2855346 overlapping records found
🔄 Step 6: Grouping and averaging ChIP signal for each gene...
✅ Averaging complete: 31433 genes with signal
🔄 Step 7: Merging signal data back into gene dataset...
✅ Merge complete.
💾 Saving final result to 'gene_dataset_with_chip.csv'...
🎉 Done! Combined dataset saved as 'gene_dataset_with_chip.csv'


In [1]:
import pandas as pd
df_output=pd.read_csv("gene_dataset_with_chip.csv")

In [2]:
df_output.head()

,gene_id,chrom,start,end,strand,alias_id,tpm_value,sequence,Chromosome,Start,End,GeneID,avg_chip_signal
0,gene:Zm00001eb000020,1,41213,46762,0,Zm00001d027231,62.888229,"[1, 3, 1, 3, 0, 2, 2, 1, 1, 1, 2, 0, 0, 0, 3, ...",1,41213,46762,gene:Zm00001eb000020,3.764350
1,gene:Zm00001eb000050,1,108553,114382,0,Zm00001d027234,0.000000,"[3, 1, 2, 3, 3, 2, 0, 2, 3, 0, 2, 1, 2, 2, 0, ...",1,108553,114382,gene:Zm00001eb000050,2.254224
2,gene:Zm00001eb000060,1,188558,189581,0,Zm00001d027239,38.358540,"[3, 3, 0, 1, 1, 2, 3, 3, 3, 1, 0, 2, 3, 0, 3, ...",1,188558,189581,gene:Zm00001eb000060,1.085869
3,gene:Zm00001eb000070,1,190191,198832,0,Zm00001d027240,4.318454,"[0, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, ...",1,190191,198832,gene:Zm00001eb000070,1.717403
4,gene:Zm00001eb000080,1,200261,203393,0,Zm00001d027242,42.850347,"[1, 1, 2, 2, 0, 3, 3, 0, 1, 3, 1, 1, 1, 0, 1, ...",1,200261,203393,gene:Zm00001eb000080,2.193772


In [3]:
import pandas as pd

# Load the full CSV file
df = pd.read_csv("gene_dataset_with_chip.csv")

# Drop redundant or duplicate columns
df_cleaned = df.drop(columns=["Chromosome", "Start", "End", "GeneID"])




In [4]:
# Optional: rename 'gene_id' if needed (e.g., remove 'gene:' prefix)
df_cleaned['gene_id'] = df_cleaned['gene_id'].str.replace('gene:', '', regex=False)

# Display a preview
print(df_cleaned.head())

           gene_id  chrom   start     end  strand        alias_id  tpm_value  \
0  Zm00001eb000020      1   41213   46762       0  Zm00001d027231  62.888229   
1  Zm00001eb000050      1  108553  114382       0  Zm00001d027234   0.000000   
2  Zm00001eb000060      1  188558  189581       0  Zm00001d027239  38.358540   
3  Zm00001eb000070      1  190191  198832       0  Zm00001d027240   4.318454   
4  Zm00001eb000080      1  200261  203393       0  Zm00001d027242  42.850347   

                                            sequence  avg_chip_signal  
0  [1, 3, 1, 3, 0, 2, 2, 1, 1, 1, 2, 0, 0, 0, 3, ...         3.764350  
1  [3, 1, 2, 3, 3, 2, 0, 2, 3, 0, 2, 1, 2, 2, 0, ...         2.254224  
2  [3, 3, 0, 1, 1, 2, 3, 3, 3, 1, 0, 2, 3, 0, 3, ...         1.085869  
3  [0, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, ...         1.717403  
4  [1, 1, 2, 2, 0, 3, 3, 0, 1, 3, 1, 1, 1, 0, 1, ...         2.193772  


In [5]:
df_cleaned.head()

,gene_id,chrom,start,end,strand,alias_id,tpm_value,sequence,avg_chip_signal
0,Zm00001eb000020,1,41213,46762,0,Zm00001d027231,62.888229,"[1, 3, 1, 3, 0, 2, 2, 1, 1, 1, 2, 0, 0, 0, 3, ...",3.764350
1,Zm00001eb000050,1,108553,114382,0,Zm00001d027234,0.000000,"[3, 1, 2, 3, 3, 2, 0, 2, 3, 0, 2, 1, 2, 2, 0, ...",2.254224
2,Zm00001eb000060,1,188558,189581,0,Zm00001d027239,38.358540,"[3, 3, 0, 1, 1, 2, 3, 3, 3, 1, 0, 2, 3, 0, 3, ...",1.085869
3,Zm00001eb000070,1,190191,198832,0,Zm00001d027240,4.318454,"[0, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 0, 0, ...",1.717403
4,Zm00001eb000080,1,200261,203393,0,Zm00001d027242,42.850347,"[1, 1, 2, 2, 0, 3, 3, 0, 1, 3, 1, 1, 1, 0, 1, ...",2.193772


In [6]:
import json
df_cleaned["sequence"] = df_cleaned["sequence"].apply(json.loads)


In [7]:
df_cleaned["sequence"].apply(len)

0        3000
1        3000
2        3000
3        3000
4        3000
         ... 
32281    3000
32282    3000
32283    3000
32284    3000
32285    3000
Name: sequence, Length: 32286, dtype: int64

In [8]:
import numpy as np
import ast
from sklearn.model_selection import StratifiedShuffleSplit
from collections import Counter


# Step 2: Normalize ChIP-seq signal
df_cleaned["avg_chip_signal"] = df_cleaned["avg_chip_signal"] / df_cleaned["avg_chip_signal"].max()

# Step 3: Prepare X: sequence + chip signal
X_seq = np.array(df_cleaned["sequence"].tolist())  # shape: (N, 3000)
chip = df_cleaned["avg_chip_signal"].values.reshape(-1, 1)  # shape: (N, 1)
chip_tiled = np.repeat(chip, X_seq.shape[1], axis=1)  # shape: (N, 3000)
X_combined = np.concatenate([X_seq, chip_tiled], axis=1)  # shape: (N, 6000)

# Step 4: Categorize TPM values
tpm_values = df_cleaned["tpm_value"].values
mean_tpm = np.mean(tpm_values)
low_threshold = mean_tpm / 2
high_threshold = mean_tpm * 1.5

def categorize_tpm(tpm, low, high):
    if tpm < low:
        return 0  # Low expression
    elif tpm < high:
        return 1  # Medium expression
    else:
        return 2  # High expression

categorized_labels = np.array([categorize_tpm(t, low_threshold, high_threshold) for t in tpm_values])
categorized_labels = categorized_labels.astype(np.int32)

# Step 5: Stratified Train-Test Split
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_idx, test_idx in splitter.split(X_combined, categorized_labels):
    x_train, x_test = X_combined[train_idx], X_combined[test_idx]
    y_train, y_test = categorized_labels[train_idx], categorized_labels[test_idx]

# Step 6: Save the arrays
np.save("x_train_zea.npy", x_train)
np.save("x_test_zea.npy", x_test)
np.save("y_train_zea.npy", y_train)
np.save("y_test_zea.npy", y_test)

# ✅ Print class distributions
print("Train distribution:", Counter(y_train))
print("Test distribution:", Counter(y_test))
print("✅ Data saved. Shapes → x_train:", x_train.shape, ", y_train:", y_train.shape)


Train distribution: Counter({0: 17636, 1: 4829, 2: 3363})
Test distribution: Counter({0: 4409, 1: 1208, 2: 841})
✅ Data saved. Shapes → x_train: (25828, 6000) , y_train: (25828,)


In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Input layer
input_layer = tf.keras.Input(shape=(6000, 1))
x = layers.Conv1D(64, 5, activation='relu')(input_layer)
x = layers.MaxPooling1D(2)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
output = layers.Dense(3, activation='softmax')(x)


# Define the model
model = models.Model(inputs=input_layer, outputs=output)

# Compile the model for classification
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary
model.summary()


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 6000, 1)]         0         
                                                                 
 conv1d (Conv1D)             (None, 5996, 64)          384       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 2998, 64)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 191872)            0         
                                                                 
 dense_6 (Dense)             (None, 128)               24559744  
                                                                 
 dense_7 (Dense)             (None, 3)                 387       
                                                           

In [20]:
from collections import Counter
print(Counter(y_train))
print(Counter(y_test))


Counter({0: 17636, 1: 4829, 2: 3363})
Counter({0: 4409, 1: 1208, 2: 841})


In [ ]:
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=6, batch_size=64)


Epoch 1/6
404/404 [==============================] - 327s 805ms/step - loss: 1.8416 - accuracy: 0.6387 - val_loss: 0.8857 - val_accuracy: 0.5850
Epoch 2/6
404/404 [==============================] - 394s 976ms/step - loss: 0.7499 - accuracy: 0.6951 - val_loss: 0.8728 - val_accuracy: 0.6798
Epoch 3/6
404/404 [==============================] - 350s 867ms/step - loss: 0.6334 - accuracy: 0.7486 - val_loss: 0.8992 - val_accuracy: 0.6384
Epoch 4/6
325/404 [=======================>......] - ETA: 1:03 - loss: 0.5163 - accuracy: 0.8036

In [19]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test MAE: {loss},{accuracy}")


202/202 [==============================] - 2s 8ms/step - loss: 0.9146 - accuracy: 0.5828
Test MAE: 0.9146071672439575,0.582843005657196


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import math

def get_positional_encoding(seq_len, model_dim):
    angle_rads = tf.range(seq_len, dtype=tf.float32)[:, tf.newaxis] / tf.pow(
        10000.0, (2 * (tf.range(model_dim, dtype=tf.float32)[tf.newaxis, :] // 2)) / model_dim
    )
    angle_rads[:, 0::2] = tf.math.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = tf.math.cos(angle_rads[:, 1::2])
    return angle_rads[tf.newaxis, ...]

class SinusoidalEmbedding(layers.Layer):
    def __init__(self, model_dim):
        super().__init__()
        self.model_dim = model_dim

    def call(self, x):
        half_dim = self.model_dim // 2
        freqs = tf.exp(tf.linspace(tf.math.log(1.0), tf.math.log(1000.0), half_dim))
        x = tf.cast(x, tf.float32)
        angles = 2.0 * math.pi * x * freqs
        sin = tf.sin(angles)
        cos = tf.cos(angles)
        return tf.concat([sin, cos], axis=-1)[..., tf.newaxis]

class TransformerBlock(layers.Layer):
    def __init__(self, model_dim, heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=heads, key_dim=model_dim)
        self.ffn = tf.keras.Sequential([
            layers.Dense(ff_dim, activation='gelu'),
            layers.Dense(model_dim),
        ])
        self.norm1 = layers.LayerNormalization(epsilon=1e-6)
        self.norm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, x, training=False):
        attn_output = self.att(x, x)
        out1 = self.norm1(x + self.dropout1(attn_output, training=training))
        ffn_output = self.ffn(out1)
        return self.norm2(out1 + self.dropout2(ffn_output, training=training))

def build_diffusion_transformer(seq_len=3000, model_dim=128, num_heads=4, ff_dim=256, depth=4):
    dna_input = tf.keras.Input(shape=(seq_len,), dtype=tf.int32)

    embedding = layers.Embedding(input_dim=5, output_dim=model_dim)(dna_input)
    pos_encoding = get_positional_encoding(seq_len, model_dim)
    x = embedding + pos_encoding[:, :seq_len, :]

    noise_var = tf.zeros((tf.shape(x)[0], 1))
    time_emb = SinusoidalEmbedding(model_dim)(noise_var)
    time_emb = tf.transpose(time_emb, [0, 2, 1])
    time_emb = tf.tile(time_emb, [1, tf.shape(x)[1], 1])
    x = tf.cast(x + time_emb, dtype=tf.float32)

    for _ in range(depth):
        x = TransformerBlock(model_dim, num_heads, ff_dim)(x)

    x = layers.GlobalAveragePooling1D()(x)
    output = layers.Dense(1)(x)  # Regression output (TPM)

    return tf.keras.Model(inputs=dna_input, outputs=output, name="DiffusionTransformer")


In [20]:
import tensorflow as tf
from tensorflow.keras import layers

class HGNNConv(tf.keras.layers.Layer):
    def __init__(self, input_dim, output_dim):
        super(HGNNConv, self).__init__()
        self.weight = self.add_weight(
            shape=(input_dim, output_dim),
            initializer='glorot_uniform',
            trainable=True
        )
        self.bias = self.add_weight(
            shape=(output_dim,),
            initializer='zeros',
            trainable=True
        )

    def call(self, x, G):
        # Apply linear transformation
        x = tf.matmul(x, self.weight) + self.bias
        # Apply hypergraph propagation
        return tf.matmul(G, x)

class HGNNEmbedding(tf.keras.Model):
    def __init__(self, input_dim=5, hidden_dim=128, dropout_rate=0.5):
        super(HGNNEmbedding, self).__init__()
        self.hgc1 = HGNNConv(input_dim, hidden_dim)
        self.hgc2 = HGNNConv(hidden_dim, hidden_dim)
        self.dropout_rate = dropout_rate

    def call(self, x, G, training=False):
        x = tf.nn.relu(self.hgc1(x, G))
        if training:
            x = tf.nn.dropout(x, rate=self.dropout_rate)
        x = tf.nn.relu(self.hgc2(x, G))
        return x


In [21]:
import tensorflow as tf
from tensorflow.keras import layers

# Channel Attention
class ChannelAttention(tf.keras.layers.Layer):
    def __init__(self, input_dim, reduction_ratio=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = layers.GlobalAveragePooling1D()
        self.max_pool = layers.GlobalMaxPooling1D()
        self.shared_dense1 = layers.Dense(input_dim // reduction_ratio, activation='relu')
        self.shared_dense2 = layers.Dense(input_dim)

    def call(self, x):
        avg_out = self.shared_dense2(self.shared_dense1(self.avg_pool(x)))
        max_out = self.shared_dense2(self.shared_dense1(self.max_pool(x)))
        out = tf.nn.sigmoid(avg_out + max_out)
        return x * tf.expand_dims(out, axis=1)

# Spatial Attention
class SpatialAttention(tf.keras.layers.Layer):
    def __init__(self):
        super(SpatialAttention, self).__init__()
        self.conv = layers.Conv1D(1, kernel_size=7, padding='same', activation='sigmoid')

    def call(self, x):
        avg_out = tf.reduce_mean(x, axis=2, keepdims=True)
        max_out = tf.reduce_max(x, axis=2, keepdims=True)
        concat = tf.concat([avg_out, max_out], axis=2)
        attention = self.conv(concat)
        return x * attention

# Cross Attention (simplified)
class CrossAttention(tf.keras.layers.Layer):
    def __init__(self, dim):
        super(CrossAttention, self).__init__()
        self.query_proj = layers.Dense(dim)
        self.key_proj = layers.Dense(dim)
        self.value_proj = layers.Dense(dim)

    def call(self, q_input, kv_input):
        Q = self.query_proj(q_input)
        K = self.key_proj(kv_input)
        V = self.value_proj(kv_input)

        scores = tf.matmul(Q, K, transpose_b=True) / tf.math.sqrt(tf.cast(tf.shape(K)[-1], tf.float32))
        attn_weights = tf.nn.softmax(scores, axis=-1)
        context = tf.matmul(attn_weights, V)
        return context

# Hybrid Fusion Module
class HybridFusion(tf.keras.Model):
    def __init__(self, input_dim, fusion_dim=128):
        super(HybridFusion, self).__init__()
        self.channel_attn = ChannelAttention(input_dim)
        self.spatial_attn = SpatialAttention()
        self.cross_attn = CrossAttention(input_dim)
        self.concat_fc = layers.Dense(fusion_dim, activation='relu')

    def call(self, diff_out, hgnn_out):
        # Step 1: Channel + Spatial Attention on each stream
        diff_attn = self.channel_attn(diff_out)
        diff_attn = self.spatial_attn(diff_attn)

        hgnn_attn = self.channel_attn(hgnn_out)
        hgnn_attn = self.spatial_attn(hgnn_attn)

        # Step 2: Cross Attention between streams
        diff_enhanced = self.cross_attn(diff_attn, hgnn_attn)
        hgnn_enhanced = self.cross_attn(hgnn_attn, diff_attn)

        # Step 3: Concatenate and fully connect
        combined = tf.concat([diff_enhanced, hgnn_enhanced], axis=-1)
        fused = self.concat_fc(combined)
        return fused


In [33]:
import tensorflow as tf
from tensorflow.keras import layers, Model

class DiffusionTransformer(tf.keras.Model):
    def __init__(self, input_seq_len=3000, embed_dim=128, num_heads=4, ff_dim=128, num_layers=2):
        super().__init__()
        self.embedding = layers.Embedding(input_dim=5, output_dim=embed_dim)
        self.pos_encoding = self._positional_encoding(input_seq_len, embed_dim)
        self.transformer_blocks = []
        for _ in range(num_layers):
            self.transformer_blocks.append([
                layers.LayerNormalization(),
                layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim),
                layers.LayerNormalization(),
                layers.Dense(ff_dim, activation='relu'),
                layers.Dense(embed_dim),
            ])
        self.pool = layers.GlobalAveragePooling1D()

    def _positional_encoding(self, length, depth):
        depth = depth // 2
        positions = tf.range(length, dtype=tf.float32)[:, tf.newaxis]
        depths = tf.range(depth, dtype=tf.float32)[tf.newaxis, :] / depth
        angle_rates = 1 / (10000**depths)
        angle_rads = positions * angle_rates
        pos_encoding = tf.concat([tf.sin(angle_rads), tf.cos(angle_rads)], axis=-1)
        return pos_encoding[tf.newaxis, ...]

    def call(self, x):
        x = self.embedding(x)
        x += self.pos_encoding[:, :tf.shape(x)[1], :]

        for norm1, attn, norm2, dense1, dense2 in self.transformer_blocks:
            attn_out = attn(norm1(x), norm1(x))
            x = x + attn_out
            ff_out = dense2(dense1(norm2(x)))
            x = x + ff_out

        return self.pool(x)

class HGNNEmbedding(tf.keras.Model):
    def __init__(self, input_seq_len=3000):
        super().__init__()
        self.mlp = tf.keras.Sequential([
            layers.Dense(128, activation='relu'),
            layers.Dense(128)
        ])
        self.pool = layers.GlobalAveragePooling1D()

    def call(self, x):
        x = tf.expand_dims(x, -1)  # (batch, seq_len, 1)
        x = self.mlp(x)
        return self.pool(x)

class HybridAttentionFusion(tf.keras.Model):
    def __init__(self, hidden_dim=128):
        super().__init__()
        self.attn = layers.Attention()
        self.dense = tf.keras.Sequential([
            layers.Dense(hidden_dim, activation='relu'),
            layers.Dense(1)
        ])

    def call(self, x1, x2):
        x1 = tf.expand_dims(x1, 1)  # (batch, 1, hidden)
        x2 = tf.expand_dims(x2, 1)
        context = self.attn([x1, x2])
        fused = tf.concat([x1, context], axis=-1)
        return self.dense(tf.squeeze(fused, axis=1))

class HybridGeneExpressionModel(tf.keras.Model):
    def __init__(self, input_seq_len=40000):
        super().__init__()
        self.dna_model = DiffusionTransformer(input_seq_len=input_seq_len)
        self.hgnn_model = HGNNEmbedding(input_seq_len=input_seq_len)
        self.fusion = HybridAttentionFusion()

    def call(self, inputs):
        x_dna, x_hgnn = inputs
        feat_dna = self.dna_model(x_dna)
        feat_hgnn = self.hgnn_model(x_hgnn)
        return self.fusion(feat_dna, feat_hgnn)


In [34]:
# Dummy data
import numpy as np
batch_size = 16
input_len = 3000

x_dna = np.random.randint(0, 5, size=(batch_size, input_len)).astype(np.int32)
x_hgnn = np.random.rand(batch_size, input_len).astype(np.float32)

# Model
model = HybridGeneExpressionModel(input_seq_len=input_len)
y_pred = model([x_dna, x_hgnn])
print("Output shape:", y_pred.shape)


Output shape: (16, 1)


In [37]:
# Sample dummy data
import numpy as np

batch_size = 32
input_len = 3000
num_samples = 1024

# Input 1: DNA sequence (int values 0–4)
x_dna = np.random.randint(0, 5, size=(num_samples, input_len)).astype(np.int32)

# Input 2: HGNN features (float values)
x_hgnn = np.random.rand(num_samples, input_len).astype(np.float32)

# Target: gene expression levels (regression) or class labels (classification)
# For regression:
# y = np.random.rand(num_samples, 1).astype(np.float32)

# OR, for binary classification:
y = np.random.randint(0, 2, size=(num_samples, 1)).astype(np.float32)


In [38]:
model = HybridGeneExpressionModel(input_seq_len=input_len)
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',  # for regression; use 'binary_crossentropy' for binary classification
    metrics=['accuracy']  # or ['accuracy'] for classification
)


In [ ]:
model.fit(
    x=[x_dna, x_hgnn],
    y=y,
    batch_size=batch_size,
    epochs=10,
    validation_split=0.2,
    shuffle=True
)


Epoch 1/10


In [ ]:
results = model.evaluate([x_dna, x_hgnn], y, batch_size=32)
print(f"Final Loss (MSE): {results[0]:.4f}")
print(f"Final MAE: {results[1]:.4f}")
